In [ ]:
## Importing all required packages

import requests
import json
import pandas as pd
import re
from yarl import URL
import numpy as np
import validators
import graphistry
from faker import Faker

In [ ]:
## CHANGE HERE

## reading the csv file that is exported using the "DiscordChatExporter" application

filename = ""
#Data = pd.read_csv(filename)


In [ ]:
### Function for identifying if a url exists within a message.
def Find(string):
  
    # findall() has been used 
    # with valid conditions for urls in string
    regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
    url = re.findall(regex,string)      
    return [x[0] for x in url]



### Function to identify if its a url and returns true or false
def isUrlValid(url):
    return True if validators.url(url) else False


In [ ]:
## creating a new column which extract only the url from the message
Data['isURLValid'] = Data['Content'].apply(Find)


In [ ]:
## converting list to a string
for index in range(len(Data)):   
   Data.loc[index,'isURLValid']=''.join(Data.loc[index,'isURLValid'])
  

In [ ]:
### Creating a column to true or false if it detects a url
Data['Flag'] = Data['isURLValid'].apply(isUrlValid)


In [ ]:
### selecting records with only links
Data_links = Data[Data['Flag']==1] 


In [ ]:
Data_links["Host_name"] = Data_links["isURLValid"].apply(lambda url: URL(url))


In [ ]:
processed = Data_links.assign(
    host=Data_links.Host_name.apply(lambda url: url.host),
    #path=Data_links.content.apply(lambda url: content.path), 
    #name=Data_links.content.apply(lambda url: content.name),
    ###query=Data_links.content.apply(lambda url: content.query_string),
)
processed

In [ ]:
# Group by the columns author_name and host and get the count
group = processed.groupby(["Author","host"]).agg(count=("Host_name","count"))
group

# Get the top 5 most popular hosts for each type
sorted_group = group.sort_values(by="count", ascending=False).reset_index()
largest = sorted_group.groupby("Author").head(5).sort_values(by='Author')
largest


In [ ]:
#To use PyGraphistry, sign up on the website for a free account then register using the account you have just created:
# https://hub.graphistry.com/
    
## CHANGE HERE

graphistry.register(api=3, username='', password='')

In [ ]:
#To create a graph with PyGraphistry, we need edges and nodes. Start with creating edges:

edges = largest[["Author", "host"]]
edges

In [ ]:
## creating nodes
"""Create a node DataFrame for a specific column"""
def create_node_df(df: pd.DataFrame, col_name: str):
    nodes = (
        df[[col_name]]
        .assign(type=col_name)
        .rename(columns={col_name: "node"})
        .drop_duplicates()
    )
    return nodes

type_nodes = create_node_df(largest, "Author")
url_nodes = create_node_df(largest, "host")
nodes = pd.concat([type_nodes, url_nodes])
nodes

g = (
    graphistry
    .edges(edges, "Author", "host")
    .nodes(nodes, "node")
)

g.plot()

In [ ]:
## create different colors for different node

types = list(edges.Author.unique())

fake = Faker()
colors = [fake.color() for _ in range(len(types))]
node_color_mapping = dict(zip(types, colors))
node_color_mapping

In [ ]:
#assign different node types with different icons

node_icon_mapping = {"host": "link", "Author": "newspaper-o"}

In [ ]:
## add color mapping and node mapping to our graph
g = (
    graphistry.edges(edges, "Author", "host")
    .nodes(nodes, "node")
    .encode_point_color(
        "node", categorical_mapping=node_color_mapping, default_mapping="silver"
    )
    .encode_point_icon(
        "Author", categorical_mapping=node_icon_mapping
    )
)

g.plot()